In [26]:
import pandas as pd
import os
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
import sqlite3

## Read Data

In [23]:
departments_df =  pd.read_csv("../data/bronce/departments.csv", header=None, names=["id","department"], index_col=0)
hired_employees_df = pd.read_csv("../data/bronce/hired_employees.csv" , header=None, names=["id","name", "datetime", "department_id", "job_id"],  index_col=0)
jobs_df = pd.read_csv("../data/bronce/jobs.csv", header=None, names=["id","job"],  index_col=0)

In [24]:
hired_employees_df

,name,datetime,department_id,job_id
id,,,,
1,Harold Vogt,2021-11-07T02:48:42Z,2.0,96.0
2,Ty Hofer,2021-05-30T05:43:46Z,8.0,NaN
3,Lyman Hadye,2021-09-01T23:27:38Z,5.0,52.0
4,Lotti Crowthe,2021-10-01T13:04:21Z,12.0,71.0
5,Gretna Lording,2021-10-10T22:22:17Z,6.0,80.0
...,...,...,...,...
1995,Goran Yong,2021-08-22T21:57:53Z,2.0,83.0
1996,Cirstoforo Youings,2021-04-01T17:48:42Z,3.0,23.0
1997,Wilek Yurkevich,2021-05-04T12:19:50Z,2.0,132.0


## EDA

In [40]:
import matplotlib.pyplot as plt
import seaborn as sns

In [48]:
hired_employees_df[hired_employees_df.isna()==True]

,name,datetime,department_id,job_id
id,,,,
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN
...,...,...,...,...
1995,NaN,NaN,NaN,NaN
1996,NaN,NaN,NaN,NaN
1997,NaN,NaN,NaN,NaN


In [27]:
def create_database_and_table(db_name, table_name, df):
    """
    Create an SQLite database and table based on DataFrame columns.

    :param db_name: str, name of the SQLite database file
    :param table_name: str, name of the table to create
    :param df: pd.DataFrame, DataFrame with columns to define the table schema
    """
    conn = sqlite3.connect(db_name)
    df.head(0).to_sql(table_name, conn, if_exists='replace', index=False)  # Create table
    conn.close()

In [28]:
def insert_data_in_batches(db_name, table_name, df, batch_size=1000):
    """
    Insert data into an SQLite table in batches.

    :param db_name: str, name of the SQLite database file
    :param table_name: str, name of the table to insert data into
    :param df: pd.DataFrame, DataFrame containing the data to insert
    :param batch_size: int, number of rows per batch
    """
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    for start in range(0, len(df), batch_size):
        batch_df = df[start:start + batch_size]
        batch_df.to_sql(table_name, conn, if_exists='append', index=False)
        conn.commit()  # Commit after each batch

    conn.close()


    

In [29]:
db_name = 'historical_data.db'
table_name = 'jobs'

In [30]:
create_database_and_table(db_name, table_name, jobs_df)
insert_data_in_batches(db_name, table_name, jobs_df, batch_size=1000)

In [34]:
conn = sqlite3.connect(db_name)


In [38]:
query = "SELECT * FROM jobs LIMIT 10"
df_from_sql = pd.read_sql_query(query, conn)
df_from_sql

,job
0,Marketing Assistant
1,VP Sales
2,Biostatistician IV
3,Account Representative II
4,VP Marketing
5,Environmental Specialist
6,Software Consultant
7,Office Assistant III
8,Information Systems Manager
9,Desktop Support Technician
